In [1]:
import os
import sys
root = os.path.dirname(os.getcwd())
src_path = os.path.join(root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)
    
from importlib import reload
import train, io_load
# reload(train)

In [ ]:
from io_load import *
# test_set = load_df_from_pq(os.path.join(root, "data_work/processed/ROI_test_set.parquet"))
test_set_w_netprofit = load_df_from_pq(os.path.join(root, "data_work/processed/ROI_test_set_w_netprofit.parquet"))


pq loaded from c:\Users\20878\25Fall\meituan-coupon-roi\data_work/processed/ROI_test_set.parquet, shape=(1966500, 62), size=91.77 MB
pq loaded from c:\Users\20878\25Fall\meituan-coupon-roi\data_work/processed/ROI_test_set_w_netprofit.parquet, shape=(1966500, 63), size=92.40 MB


In [6]:
num_receipts = test_set_w_netprofit.shape[0]
sum_profit = sum(test_set_w_netprofit["Actual_pay_cent"])
print("the net profit each coupon receipt brings in:", sum_profit/num_receipts)

the net profit each coupon receipt brings in: 131.8117491736588


In [6]:
test_set_w_netprofit[["label_same_user_st", "Actual_pay_cent"]].value_counts()

label_same_user_st  Actual_pay_cent
0                   0.0                1891526
1                   1700.0                 413
                    2000.0                 408
                    1800.0                 380
                    1500.0                 377
                                        ...   
                    139100.0                 1
                    135200.0                 1
                    133400.0                 1
                    130700.0                 1
                    38.0                     1
Name: count, Length: 8176, dtype: int64

In [7]:
set(test_set_w_netprofit["receipt_key"].unique()) == set(test_set["receipt_key"].unique())

True

In [2]:
from io_load import *
from train import *
import pandas as pd
test_pred_w_profit =pd.read_csv(os.path.join(root, "experiment_results/ROI/test_pred_ROI_0115_m_7d.csv"))
test_pred_w_profit.head()

,y_true,Actual_pay_cent,y_pred
0,1,1920.0,0.677465
1,1,1480.0,0.672333
2,0,0.0,0.408591
3,0,0.0,0.385008
4,0,0.0,0.422859


In [3]:
test_pred_w_profit.shape

(1966500, 3)

In [4]:
print("individual class accuracy:")
print("Recall:", metric_individual_class_accuracy(y_true=test_pred_w_profit["y_true"],
                                       y_pred=test_pred_w_profit["y_pred"]))
print("True Negative Rate:", metric_individual_class_accuracy(
    y_true=test_pred_w_profit["y_true"],
    y_pred=test_pred_w_profit["y_pred"],
    which_class=0))
print("----------------------------------------------")

print("business metrics:")
top5redem, top5profit = metric_business_top5profit(
    y_true=test_pred_w_profit["y_true"],
    y_pred=test_pred_w_profit["y_pred"],
    profit=test_pred_w_profit["Actual_pay_cent"]
)
print("redemption rate of the coupons with the top 5 percent raw prediction score:", top5redem)
print("net profit per coupon with top 5 percent prediction score brings in:", top5profit)

profit_at_diff_thrshld = metric_business_profit_at_diff_thrshld(
    y_pred=test_pred_w_profit["y_pred"],
    profit=test_pred_w_profit["Actual_pay_cent"] 
)
print("the net profit each predicted-as-redeemed coupon brings in under different thresholds:")
print(profit_at_diff_thrshld)

individual class accuracy:
Recall: [np.float64(1.0), np.float64(0.768), np.float64(0.0)]
True Negative Rate: [np.float64(0.0), np.float64(0.667), np.float64(1.0)]
----------------------------------------------
business metrics:
redemption rate of the coupons with the top 5 percent raw prediction score: 0.18892817107872725
net profit per coupon with top 5 percent prediction score brings in: 487.2355314592854
the net profit each predicted-as-redeemed coupon brings in under different thresholds:
[np.float64(131.81174917365877), np.float64(251.47887651894453), 0]
